# 2019 OK JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: I11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:

* Do I need to cite things in-code? nah, but track examples/ code snippets
* Do we need a maximum difference function or class? function
* check data to stop kernel from dying

In [1]:
%matplotlib inline

import sys
import numpy as np
import astropy as ap
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import QTable
import matplotlib.pyplot as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons
import pyoorb as oo
oo.pyoorb.oorb_init()

0

# Mary Berry ready Zone

In [2]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:01'
internal_start = ap.time.Time(start_time)
internal_stop = ap.time.Time(stop_time)
element_time = internal_start.jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

## JPL Values

In [3]:
# 2019 OK elements
el_obj = Horizons(id=obj_id, location= '500@10',
               epochs=element_time)
el_jpl = el_obj.elements()

In [4]:
# 2019 OK JPL ephem
ephem_obj = Horizons(id= obj_id, location= Gemini_S,
               epochs={'start': start_time, 'stop':stop_time,
                      'step':'1d'})
ephem_jpl = ephem_obj.ephemerides()

## OpenOrb-Sourced Values

In [5]:
# time conversions, epochs for pyoorb to work
element_time_pyoorb = internal_start.mjd
start_pyoorb = internal_start.mjd
stop_pyoorb = internal_stop.mjd
peri_time = ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd

In [ ]:
# 2019 OK orbit and time set-up
orbit_2019OK = np.array([[0, el_jpl['q'][0], el_jpl['e'][0], np.deg2rad(el_jpl['incl'][0]),
                   np.deg2rad(el_jpl['Omega'][0]), np.deg2rad(el_jpl['w'][0]),
                   peri_time, 2, start_pyoorb, 1, el_jpl['H'][0], el_jpl['G'][0]]],
                  dtype=np.double, order='F')

t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')


In [ ]:
ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=orbit_2019OK,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')

print(ephem_pyoorb, err)

In [6]:
# function to reorganize JPL Horizons output into pyoorb-acceptable inputs. Expand for multiple orbits next
def reorganizer(orbits, epoch):
    '''
    Parameters
    ------
    orbits : `~numpy.ndarray` (N, 18)
    epoch : `~numpy.ndarray` (3652, 2)
        Constrained to cometary format.
    Returns
    -------
    new_array : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    '''
    
    temp = orbits.copy()
    temp = temp.as_array().data
    if temp.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = temp.shape[0]
        
    for i in range(num_orbits):
        ids = i
        orbit_type = 2
        time_scale = 3
        
    # elements x6
    q = temp[0][6]
    e = temp[0][5]
    incl = np.deg2rad(temp[0][7])
    longnode = np.deg2rad(temp[0][8])
    argper = np.deg2rad(temp[0][9])
    peri_epoch = ap.time.Time(temp[0][10], format='jd').mjd

    mag = temp[0][3]
    slope = temp[0][4]
    
    if num_orbits > 1:
        new_array = np.array(
            np.array([
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]), 
            dtype=np.double, 
            order='F')
    else:
        new_array = np.array(
            [[
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]], 
            dtype=np.double,
            order='F')
    
    return new_array

In [8]:
hi = reorganizer(el_jpl, start_pyoorb)
t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=hi,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')

print(ephem_pyoorb, err)

[[[ 5.51970000e+04  2.30986909e+02 -1.98286587e+01 ...  3.98992783e+00
    2.96949445e+01  1.75134076e+02]
  [ 5.51980000e+04  2.31224798e+02 -1.98898480e+01 ...  3.97929769e+00
    2.96954430e+01  1.75211477e+02]
  [ 5.51990000e+04  2.31460894e+02 -1.99503260e+01 ...  3.96850089e+00
    2.96957376e+01  1.75288822e+02]
  ...
  [ 5.88470000e+04  2.11597036e+02 -1.45598625e+01 ...  1.97565879e+00
    2.73085098e+01  1.36170577e+02]
  [ 5.88480000e+04  2.11881736e+02 -1.46758459e+01 ...  1.97297843e+00
    2.73183593e+01  1.36440068e+02]
  [ 5.88490000e+04  2.12160998e+02 -1.47896417e+01 ...  1.97013600e+00
    2.73278164e+01  1.36706619e+02]]] 0


## Difference metric

In [9]:
# difference function: how to write it so that any integrator we use can be parsed?

# use great circle difference, keep coordinates and magnitude

def max_diff(ephem1, ephem2):
    '''
    Parameters
    ----------
    ephem1: elements of ephemerides, 1st integrator, numpy array?
    ephem2: elements of ephemerides, 2nd integrator, numpy array? (same type as ephem1)
    
    Returns
    -------
    diff_matrix: separation, offset, magnitude diff between ephem1 and ephem2, numpy array
    '''
    
    #coordinates
    RA1 = ephem1['RA']
    Dec1 = ephem1['DEC']
    RA2 = ephem2[0][:,1] * u.deg
    Dec2 = ephem2[0][:,2] * u.deg
    
    coord1 = SkyCoord(RA1, Dec1, frame='icrs')
    coord2 = SkyCoord(RA2, Dec2, frame='icrs')
    sep = coord1.separation(coord2)
    
    # magnitudes
    #mag1 = ephem1['H']
    #mag2 = ephem2['H']
    
    #magnitude = mag2 - mag1
    
    # matrix
    diff_matrix = np.array([sep])
        
    return diff_matrix

In [10]:
great_circle_diff = max_diff(ephem_jpl, ephem_pyoorb)
print(great_circle_diff)

[[1.61263632e-06 2.72481735e-06 5.76005645e-06 ... 2.98948516e-04
  3.02306369e-04 2.97839368e-04]]


# Recipe

In [ ]:
'''
Gfortran version 4.8.6

Useful bits from docs:

JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

pyoorb:
orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target


JPL TableColumns values to extract:
[0,1,2,3,7,8,23] = [targetname, datetime str, datettime jd, H, RA, DEC, V]

Want pyoorb.oorb_ephemeris_basic , use these indices for properties:
[0,1,2,9] = [mjd, RA (deg), DEC(deg), predicted apparent V-band mag]

example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
'''

In [ ]:
# copied from Joachim's Thor repo
import warnings
import os

def _configureOrbitsPYOORB(orbits, t0, orbit_type,  time_scale, magnitude, slope):
    """
    Convert an array of orbits into the format expected by PYOORB. 
    
    Parameters
    ----------
    orbits : `~numpy.ndarray` (N, 6)
        Orbits to convert. See orbit_type for expected input format.
    t0 : `~numpy.ndarray` (N)
        Epoch in MJD at which the orbits are defined. 
    orbit_type : {'cartesian', 'keplerian', 'cometary'}, optional
        Orbital element representation of the provided orbits. 
        If cartesian:
            x : heliocentric ecliptic J2000 x position in AU
            y : heliocentric ecliptic J2000 y position in AU
            z : heliocentric ecliptic J2000 z position in AU
            vx : heliocentric ecliptic J2000 x velocity in AU per day
            vy : heliocentric ecliptic J2000 y velocity in AU per day
            vz : heliocentric ecliptic J2000 z velocity in AU per day
        If keplerian:
            a : semi-major axis in AU
            e : eccentricity in degrees
            i : inclination in degrees
            Omega : longitude of the ascending node in degrees
            omega : argument of periapsis in degrees
            M0 : mean anomaly in degrees
        If cometary:
            p : perihelion distance in AU
            e : eccentricity in degrees
            i : inclination in degrees
            Omega : longitude of the ascending node in degrees
            omega : argument of periapsis in degrees
            T0 : time of perihelion passage in MJD
    time_scale : {'UTC', 'UT1', 'TT', 'TAI'}, optional
        Time scale of the MJD epochs.
    magnitude : float or `~numpy.ndarray` (N), optional
        Absolute H-magnitude or M1 magnitude. 
    slope : float or `~numpy.ndarray` (N), optional
        Photometric slope parameter G or K1.
    Returns
    -------
    orbits_pyoorb : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    """
    orbits_ = orbits.copy()
    if orbits_.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = orbits_.shape[0]

    if orbit_type == "cartesian":
        orbit_type = [1 for i in range(num_orbits)]
    elif orbit_type == "cometary":
        orbit_type = [2 for i in range(num_orbits)]
        orbits_[:, 2:5] = np.radians(orbits_[:, 2:5])
    elif orbit_type == "keplerian":
        orbit_type = [3 for i in range(num_orbits)]
        orbits_[:, 1:] = np.radians(orbits_[:, 1:])
    else:
        raise ValueError("orbit_type should be one of {'cartesian', 'keplerian', 'cometary'}")

    if time_scale == "UTC":
        time_scale = [1 for i in range(num_orbits)]
    elif time_scale == "UT1": 
        time_scale = [2 for i in range(num_orbits)]
    elif time_scale == "TT":
        time_scale = [3 for i in range(num_orbits)]
    elif time_scale == "TAI":
        time_scale = [4 for i in range(num_orbits)]
    else:
        raise ValueError("time_scale should be one of {'UTC', 'UT1', 'TT', 'TAI'}")

    if type(slope) != np.ndarray:
        slope = [slope for i in range(num_orbits)]
    if type(magnitude) != np.ndarray:
        magnitude = [magnitude for i in range(num_orbits)]
    if type(t0) != np.ndarray:
        t0 = [t0 for i in range(num_orbits)]

    ids = [i for i in range(num_orbits)]

    if num_orbits > 1:
        orbits_pyoorb = np.array(
            np.array([
                ids, 
                *list(orbits_.T), 
                 orbit_type, 
                 t0, 
                 time_scale, 
                 magnitude, 
                 slope
            ]).T, 
            dtype=np.double, 
            order='F'
        )
    else:
        orbits_pyoorb = np.array(
            [[
                ids[0], 
                *list(orbits_.T), 
                orbit_type[0], 
                t0[0], 
                time_scale[0], 
                magnitude[0], 
                slope[0]]
            ], 
            dtype=np.double,
            order='F')
        
    return orbits_pyoorb

def _configureEpochsPYOORB(epochs, time_scale):
    """
    Convert an array of orbits into the format expected by PYOORB.
    
    Parameters
    ----------
    epochs : `~numpy.ndarray` (N)
        Epoch in MJD to convert. 
    time_scale : {'UTC', 'UT1', 'TT', 'TAI'} 
        Time scale of the MJD epochs.
        
    Returns
    -------
    epochs_pyoorb : `~numpy.ndarray (N, 2)
        Epochs converted into the PYOORB format.
    """
    num_times = len(epochs)
    if time_scale == "UTC":
        time_scale = [1 for i in range(num_times)]
    elif time_scale == "UT1": 
        time_scale = [2 for i in range(num_times)]
    elif time_scale == "TT":
        time_scale = [3 for i in range(num_times)]
    elif time_scale == "TAI":
        time_scale = [4 for i in range(num_times)]
    else:
        raise ValueError("time_scale should be one of {'UTC', 'UT1', 'TT', 'TAI'}")
    
    epochs_pyoorb = np.array(list(np.vstack([epochs, time_scale]).T), dtype=np.double, order='F')
    return epochs_pyoorb

In [ ]:
el_jpl.as_array().data